## Building a Chatbot :

- Design and implement an LLM-powered chatbot which will able to have conversation and remember previous interaction.
- This chatbot that will only use the language model to have a conversation.
- Few other related concepts are available :
    - Conversational RAG : Chat with external or private data
    - Agent : A chatbot can take a action.

In [44]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY = os.getenv("GROQ_API_KEY")

In [72]:
from langchain_groq import ChatGroq
# model = ChatGroq(model="gemma2-9b-it", groq_api_key=GROQ_API_KEY)
# model = ChatGroq(model="gemma-7b-it", groq_api_key=GROQ_API_KEY)
model = ChatGroq(model="llama-3.1-8b-instant", groq_api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f31088f37c0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f31088f5000>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [73]:
from langchain_core.messages import HumanMessage
model.invoke([HumanMessage(content="Hi, I am Pranav and I am GCP Cloud AIML Engineer.")])

AIMessage(content="Nice to meet you, Pranav. As a GCP Cloud AI ML Engineer, you must be working with Google Cloud Platform's AI and machine learning services. That's a cutting-edge field with a lot of exciting opportunities. \n\nWhat specific areas of AI and ML are you currently working on or interested in? Are you using services like TensorFlow, AutoML, or Vertex AI? Or perhaps you're exploring other GCP services like Cloud Dataflow or Cloud Storage? Let's chat about your work!", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 52, 'total_tokens': 154, 'completion_time': 0.136, 'prompt_time': 0.010481001, 'queue_time': 0.004025859, 'total_time': 0.146481001}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_f66ccb39ec', 'finish_reason': 'stop', 'logprobs': None}, id='run-7faa25ba-f617-44d2-bafc-5e1f311d8de5-0', usage_metadata={'input_tokens': 52, 'output_tokens': 102, 'total_tokens': 154})

In [74]:
# Example of the Chatbot
# made a hardcode of HumanMessage and AIMessage to see when next HumanMessage will come how AIMessage will behave

from langchain_core.messages import AIMessage
model.invoke(
    [
        HumanMessage(content="Hi, I am Pranav and I am GCP Cloud AIML Engineer."),
        AIMessage(content="Hi Pranav, it's nice to meet you! \n\nAs a GCP Cloud AIML Engineer, you must be working on some exciting projects. \n\nWhat kind of work are you most passionate about in the field of AIML?  Do you have any particular areas of expertise or projects you'd like to share?"),
        HumanMessage(content="Hey, what's my name and what do I do ?")
    ]
)

AIMessage(content='Your name is Pranav, and you are a GCP Cloud AIML Engineer.', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 139, 'total_tokens': 158, 'completion_time': 0.025333333, 'prompt_time': 0.02837531, 'queue_time': 0.004472137000000001, 'total_time': 0.053708643}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-4d7d0cbc-cc7e-4d8f-b748-fe46ece072bd-0', usage_metadata={'input_tokens': 139, 'output_tokens': 19, 'total_tokens': 158})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [75]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store ={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    '''
    session_id will get inherited (use) to BaseChatMessageHistory to create a difference between sessions.
    '''
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model, get_session_history)

In [76]:
configurations={"configurable":{"session_id":"chat"}}

In [77]:
response = with_message_history.invoke(
    [HumanMessage(content="Hi, I am Pranav and I am GCP Cloud AIML Engineer working in Cognizant where helping the GCP customers for POCs.")],
    config=configurations
)

In [78]:
response.content

"Nice to meet you, Pranav. It's great that you're working as a Cloud AI ML Engineer in Cognizant, specifically helping GCP customers with Proof of Concepts (POCs). AI and ML are rapidly evolving fields, and Google Cloud Platform (GCP) provides a wide range of tools and services to support AI and ML workloads.\n\nAs a Cloud AI ML Engineer, what kind of POCs do you usually work on? Are you working with clients on building custom AI/ML models, deploying them on GCP, or helping them integrate with other GCP services like Vertex AI, Dataflow, or BigQuery?"

In [79]:
with_message_history.invoke(
    [HumanMessage(content="Who am I?")],
    config=configurations
)

AIMessage(content='You, Pranav, are a GCP Cloud AI/ML Engineer working in Cognizant, where you help GCP customers with Proof of Concepts (POCs).', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 37, 'prompt_tokens': 213, 'total_tokens': 250, 'completion_time': 0.049333333, 'prompt_time': 0.043540418, 'queue_time': 0.004131152000000006, 'total_time': 0.092873751}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-eaf66734-8c8d-47bd-b320-9719bd2ceaf2-0', usage_metadata={'input_tokens': 213, 'output_tokens': 37, 'total_tokens': 250})

In [80]:
# change the config which mean I am chaning the "session_id"
configurations1={"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="Who am I?")],
    config=configurations1
)

response.content

"Unfortunately, I don't have any information about you. This conversation just started, and we haven't discussed anything specific yet. If you'd like to share something about yourself or ask a question, I'm here to help."

In [81]:
response = with_message_history.invoke(
    [HumanMessage(content="Hey, I am John.")],
    config=configurations1
)

response.content

"Nice to meet you, John. That's a common name, so I don't have any additional information about you specifically. Would you like to tell me more about yourself, or is there something particular you'd like to talk about?"

In [82]:
configurations1={"configurable":{"session_id":"chat1"}}

response = with_message_history.invoke(
    [HumanMessage(content="What do you know about me?")],
    config=configurations1
)

response.content

"Unfortunately, I don't know much about you, John. Since our conversation just started, I don't have any prior knowledge or personal data about you. However, I can make some educated guesses based on our conversation so far.\n\nYou:\n\n* Are a person (obviously!)\n* Have a name, John\n* Are conversing with a language model (me)\n* Are interested in getting to know me or asking questions\n\nThat's about it, though. If you'd like to share more about yourself, I'm happy to listen and learn!"

### Promp Templates :
Prompt Templates help to turn raw user information into a content format that the LLM can work. In this case, the raw user input is just a message, which we are passing to the LLM. Now let's add a system messages with some custom instructions (will take messages as input).

In [83]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a helpful assistance. Answer all the questions to the best of your ability."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [84]:
chain.invoke({"messages":[HumanMessage(content="Hi, I am Deps.")]})

AIMessage(content="Hello Deps, it's nice to meet you. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 59, 'total_tokens': 78, 'completion_time': 0.025333333, 'prompt_time': 0.015046619, 'queue_time': 0.09693327, 'total_time': 0.040379952}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_9cb648b966', 'finish_reason': 'stop', 'logprobs': None}, id='run-5efd5c52-916f-4c9c-9595-91136394389b-0', usage_metadata={'input_tokens': 59, 'output_tokens': 19, 'total_tokens': 78})

In [85]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [86]:
configurations2={"configurable":{"session_id":"chat2"}}

response = with_message_history.invoke(
    [HumanMessage(content="Hi, I am Deps.")],
    config=configurations2
)

response.content

'Nice to meet you, Deps. How can I assist you today?'

In [87]:
# Add some more complexity on top of it.

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You're a helpful assistance. Answer all the questions to the best of your ability in {language}."
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain = prompt|model

In [88]:
response = chain.invoke(
    {
        "messages":[HumanMessage(content="Hi, My name is Pranav.")],
        "language":"Hindi"
    }
)

response.content

'नमस्कार प्रानव जी, मैं आपकी सहायता करने के लिए तैयार हूँ।'

In [89]:
# No wrap this mre complicated Message History class 

with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [90]:
configurations4={"configurable":{"session_id":"chat4"}}

response=with_message_history.invoke(
    {"messages":[HumanMessage(content="Hi, I am Pranav and I am working professional AI Engineer at CTS")],
     "language":"Hindi",
    },
    config=configurations4
)

response.content

'नमस्ते प्रणव जी, आपका स्वागत है! आपके Pronav का अर्थ है सपना या आकांक्षा (Pranav)। यह एक सुंदर नाम है और यह जानने पर खुशी हुई कि आप एक पेशेवर एआई इंजीनियर हैं और CTS में काम करते हैं।\n\nक्या आपको कोई विशिष्ट समस्या है जिसका समाधान चाहिए या आपने कुछ नए विकास के बारे में जानना चाहते हैं?'

In [98]:
response=with_message_history.invoke(
    {"messages":[HumanMessage(content="What's my name?")],
     "language":"Hindi",
    },
    config=configurations4
)

response.content

'आपका नाम प्रणव है!'